In [ ]:
import pandas as pd

In [ ]:
train_dataset = pd.read_csv("train.csv")
eval_dataset = pd.read_csv("evaluation.csv")

In [1]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input

try:
    import tensorflow_hub as hub
except ModuleNotFoundError:
    %pip install tensorflow_hub
    import tensorflow_hub as hub
    
try:
    from keras_self_attention import SeqSelfAttention
except ModuleNotFoundError:
    %pip install keras-self-attention
    from keras_self_attention import SeqSelfAttention

import numpy as np


# %pip install -q transformers
# %pip install -q -U tensorflow-text
# %pip install -q tf-models-official==2.7.0

import math as m

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18912 sha256=8a7a2ce668c47971c45483a375c639b5310c09e8810fed5255427872673bf480
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-self-attention


In [2]:
huburl = "https://tfhub.dev/google/universal-sentence-encoder/4" 
fine_tuned_module_object = hub.load(huburl)

In [3]:
def gen_random_batch(in_groups, batch_halfsize = 8):
    text_batch, reason_batch, y_hat = [], [], []
    all_groups = list(range(in_groups[0].shape[0]))
    for match_group in [True, False]:
        group_idx = np.random.choice(all_groups, size = batch_halfsize)
        text_batch += [in_groups[0][c_idx] for c_idx in group_idx]
        if match_group:
            b_group_idx = group_idx
            y_hat += [1]*batch_halfsize
        else:
            # anything but the same group
            non_group_idx = [np.random.choice([i for i in all_groups if i!=c_idx]) for c_idx in group_idx] 
            b_group_idx = non_group_idx
            y_hat += [0]*batch_halfsize
            
        reason_batch += [in_groups[1][c_idx] for c_idx in b_group_idx]
            
    return np.stack(text_batch, 0), np.stack(reason_batch, 0), np.stack(y_hat, 0)

def siam_gen(in_groups, batch_size = 32):
    while True:
        text_stack, reason_stack, y_hatstack = gen_random_batch(in_groups, batch_size//2)
        yield [text_stack, reason_stack], y_hatstack


def identity_loss(y_true, y_pred):
    return K.mean(y_pred)

def triplet_loss(x, alpha = 0.2):
    # Triplet Loss function.
    anchor,positive,negative = x
    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)
    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)
    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
    return loss

In [4]:
def accuracy(y_true, y_pred):
    results = (y_pred <= 0.5).astype(int).squeeze()
    return np.mean([y_true == results])

In [5]:
tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
tfhub_handle_encoder = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/2"
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess)

FileNotFoundError: ignored

In [6]:
def encoder(name):
    inputs = Input(shape=(), dtype=tf.string)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name=name)
    
    encoder_inputs = preprocessing_layer(inputs)
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    
    a = tfl.Dense(64, activation="linear")(net)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dense(32, activation="linear")(a)
    a = tfl.BatchNormalization()(a)
    outputs = tfl.Activation('relu')(a)
    
    model = Model(inputs, outputs)
    return model

In [ ]:
def get_model():
    #textEncoder
    inputText = Input(shape=(), dtype=tf.string)
    textModel = encoder("textBertEncoder")
    text_embedd = textModel(inputText)

    #reasonEncoder
    reasonModel = encoder("reasonBertEncoder")

    inputReason = Input(shape=(), dtype=tf.string)
    reason_embedd = reasonModel(inputReason)

    #nonReasonEncoder
    inputNotReason = Input(shape=(), dtype=tf.string)
    nonreason_embedd = reasonModel(inputNotReason)

    loss = Lambda(triplet_loss)([text_embedd, reason_embedd, nonreason_embedd])

    model = Model(inputs = [inputText, inputReason, inputNotReason], outputs = [loss], name="bertModel")
    return model, textModel, reasonModel

In [ ]:
model = get_model()

NameError: name 'tf' is not defined

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam", loss = "binary_crossentropy", metrics = [adfsccuracy()])

In [ ]:
loss_history = model.fit(siam_gen([df["clean_text"], df["reason"]], 64), 
                         validation_data = siam_gen([df_test["clean_text"], df_test["reason"]], 32),
                         steps_per_epoch=50, validation_steps=10, epochs = 100, 
                         verbose = True, use_multiprocessing=True)